In [1]:
# Code By adityatb at https://github.com/adityatb/noise-reduction-using-rnn
# LSTM method test.
# Maintain by ShYy, 2018.

import scipy
import scipy.signal as signal
import numpy as np
import os, random, sys
import scipy.io.wavfile as wav
import math

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
%tensorflow_version 1.8
import tensorflow as tf

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.8`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
cd /content/gdrive/My Drive/Data/

/content/gdrive/My Drive/Data


In [5]:
os.getcwd()

'/content/gdrive/My Drive/Data'

In [6]:
#filename을 입력받아 앞에서부터 len(filename)-11 까지만 남긴 뒤 _voice 추가
def formatFilename(filename):
    return filename[:len(filename) - 11] + "_voice.wav"


# Strip away the _xnoise.wav part of the filename, and append _voice.wav to obtain clean voice counterpart

def create_final_sequence(sequence, max_length):
    a, b = sequence.shape
    extra_len = max_length - b
    null_mat = np.zeros((len(sequence), extra_len), dtype=np.float32)
    sequence = np.concatenate((sequence, null_mat), axis=1)
    return sequence


def sequentialized_spectrum(batch):
    # Get maximum length of batch
    t = []
    t_vec = []
    Sxx_Vec = []
    for each in batch:
        _, t, Sxx_Vec_Temp = signal.stft(each, fs=rate_repository[0], nperseg=stft_size, return_onesided = False)
        t_vec.append(t)
        Sxx_Vec.append(Sxx_Vec_Temp)
    maximum_length = findMaxlen(t_vec)

    max_run_total = int(math.ceil(float(maximum_length) / sequence_length))
    final_data = np.zeros([len(batch), max_run_total, stft_size, sequence_length])
    true_time = np.zeros([len(batch), max_run_total])

    # Read in a file and compute spectrum
    # for batch_idx, each_set in enumerate(batch):
    for batch_idx, Sxx in enumerate(Sxx_Vec):
        # f, t, Sxx = signal.stft(each_set, fs=rate_repository[0], nperseg=stft_size, return_onesided = False)

        # Magnitude and Phase Spectra
        Mag = Sxx.real
        t = t_vec[batch_idx]
        # Phase = Sxx.imag

        # Break up the spectrum in sequence_length sized data
        run_full_steps = float(len(t)) / sequence_length
        run_total = int(math.ceil(run_full_steps))

        # Run a loop long enough to break up all the data in the file into chunks of sequence_size
        for step in range(run_total):

            begin_point = step * sequence_length
            end_point = begin_point + sequence_length

            m, n = Mag[:, begin_point:end_point].shape

            # Store each chunk sequentially in a new array, accounting for zero padding when close to the end of the file
            if n == sequence_length:
                final_data[batch_idx, step, :, :] = np.copy(Mag[:, begin_point:end_point])
                true_time[batch_idx, step] = n
            else:
                final_data[batch_idx, step, :, :] = np.copy(create_final_sequence(Mag[:, begin_point:end_point], sequence_length))
                true_time[batch_idx, step] = n

    final_data = np.transpose(final_data, (0, 1, 3, 2))

    return final_data, true_time, maximum_length


def findMaxlen(data_vec):
    max_ = 0
    for each in data_vec:
        if len(each) > max_:
            max_ = len(each)
    return max_



In [9]:
# ----------------- Begin Vars --------------------- #

# Training data directories
traindata = os.getcwd() + "/Training/Self_NoiseAdded/"
voicedata = os.getcwd() + "/Training/HumanVoices2/"
checkpoints = os.getcwd() + "/TF_Checkpoints2/"

# NormConstant
norm_factor = (1 / 32768.0)

# Spectrogram Parameters
stft_size = 1024

# RNN Specs
sequence_length = 100
batch_size = 10
learning_rate = 0.001
epochs = 250
number_of_layers = 3

# Tensorflow vars + Graph and LSTM Params
input_data = tf.placeholder(tf.float32, [None, sequence_length, stft_size])
clean_data = tf.placeholder(tf.float32, [None, sequence_length, stft_size])
sequence_length_tensor = tf.placeholder(tf.int32, (None))

# Temp_data_variables
no_of_files = 0
temp_list = []
final_data = []
sequence_length_id = 0

# Repositories
file_repository = []
rate_repository = []
clean_repository = []

# Selected vectors
files_vec = []
clean_files_fin_vec = []
clean_files_vec = []

# Graph
lstm_cell = tf.contrib.rnn.BasicLSTMCell(stft_size, forget_bias = 1.0)
lstm_cell2 = tf.contrib.rnn.BasicLSTMCell(stft_size, forget_bias = 1.0, state_is_tuple = True)
lstm_cell3 = tf.contrib.rnn.BasicLSTMCell(stft_size, forget_bias = 1.0, state_is_tuple = True)
#lstm Cell 생성

#stacked_lstm = tf.contrib.rnn.MultiRNNCell([[lstm_cell] for i in range(number_of_layers)])
#stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell,lstm_cell2])
#stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell],state_is_tuple=True)
#층 쌓기

init_state = lstm_cell.zero_state(batch_size, tf.float32)
#초기상태를 zero로 초기화

rnn_outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, input_data, sequence_length=sequence_length_tensor, initial_state=init_state, time_major=False)
#rnn_outputs, final_state = tf.nn.static_rnn(stacked_lstm, input_data, sequence_length=sequence_length_tensor, initial_state=init_state)
#TensorFlow dynamic_rnn을 사용하면 모델의 결과와 마지막 상태값을 반환하는데, 이것들이 학습하는 동안 배치 사이에서 전달되어야 한다.

mse_loss = tf.losses.mean_squared_error(rnn_outputs, clean_data)
#rnn_output을 이용 loss함수는 mse를 사용한다.

# train_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(mse_loss)
# train_optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(mse_loss)
# train_optimizer = tf.train.AdagradDAOptimizer(learning_rate).minimize(mse_loss)
train_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(mse_loss)
#optimizer로는 Adam을 사용

saver = tf.train.Saver()
#tf.train.Saver()함수를 이용해 만들어놓은 cell 과 layer, optimizer, lossfunction 을 저장한다.

TypeError: ignored

In [ ]:
# ------------------- Read all data to memory creating a repository of mixture and clean files --------------------- #

os.chdir(traindata)
# for file_iter in range(traindata):

# Buffer training data to memory for faster execution:
for root, _, files in os.walk(traindata):
    files = sorted(files)
    no_of_files = len(files)

    #files는 NoiseAdded data들을 의미하며 배치사이즈가 training data size보다 크면 말이 안되니까 여기서 에러처리
    if batch_size > no_of_files:
        sys.exit("Error: batch_size cannot be more than number of files in the training directory")

    #files를 읽어와 data와 samplingrate를 각각 리스트에 저장
    for f in files:
        if f.endswith(".wav"):
            temp_list.append(f)
            srate, data = wav.read(os.path.join(root, f))
            file_repository.append(data)
            rate_repository.append(srate)

# Generate a vector of file names that are clean files
#map은 (함수,인자) 를 입력으로 받고 들어온 모든 인자에 대해 함수를 실행시켜 리스트형태로 반환시켜주는 함수이다 고로 들어온 temp_list 인자들에 대해 formatFilename이 실행된다
#간단히 말하면 기존의 이름을 바꾸는 역할을 한다
clean_files_vec = list(map(formatFilename, temp_list))
# clean_files_vec = list(map(None, *clean_files_vec))

# Find clean files that correspond to data in file_repository and buffer clean voice data to memory
#clean한 목소리 데이터들을 clean_repasitory에 저장
for root, _, files in os.walk(voicedata):
    files = sorted(files)
    for each in files:
        if each.endswith(".wav"):
            for name in clean_files_vec:
                if each == name:
                    srate2, data2 = wav.read(os.path.join(root, name))
                    clean_repository.append(data2)

"""
#최종적으로
file_repository
rate_repository
clean_repository
가 생성됨
"""

In [ ]:
# files_vec = []
run_epochs = int((no_of_files / batch_size) * epochs)
print(run_epochs)
#no_of_files는 training 파일의 개수이니 3000
#batch_size는 10으로 설정됨
#epochs 는 250 이므로
#run_epochs 의 값은 300 * 250 = 75000

In [ ]:
# ------------------- Step 1: Prepare data in batches and perform STFTs --------------------- #
# Initialize TF Graph
init_op = tf.global_variables_initializer()  # initialize_all_variables()
gpu_options = tf.GPUOptions(allow_growth = True)            # Set session GPU using growing.

sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))
sess.run(init_op)

globalBatchLossSum = 0          # Sum of all batch losses
globalStepsSum = 0          # Sum of all steps
lastCumulativeLossAvg = 100           # Last Cumulative Loss Avg.

for idx in range(int(run_epochs)):

    files_vec = []
    # clean_files_vec = []
    clean_files_fin_vec = []

    # Select batch_size no. of random number of files from file_repository and the corresponding clean files
    for file_iter in range(batch_size):
        i = random.randint(0, len(file_repository) - 1)
        files_vec.append(file_repository[i] * norm_factor)
        clean_files_fin_vec.append(clean_repository[i] * norm_factor)

    stft_batch, sequence_length_id, maximum_length = sequentialized_spectrum(files_vec)
    clean_voice_batch, sequence_length_id_clean, maximum_length_clean = sequentialized_spectrum(clean_files_fin_vec)

    # ------------------- Step 2: Feed Data to Placeholders, and then, Initialise, Train and Save the Graph  --------------------- #

    max_time_steps = stft_batch.shape[1]
    batchLossSum = 0            # Sum of batch losses in one index.

    for time_seq in range(max_time_steps):
        feed_dict = {
            input_data: stft_batch[:, time_seq, :, :],
            clean_data: clean_voice_batch[:, time_seq, :, :],
            sequence_length_tensor: sequence_length_id[:, time_seq]
        }
        _, loss_value, final_state_value, rnn_outputs_val = sess.run([train_optimizer, mse_loss, final_state, rnn_outputs], feed_dict=feed_dict)

        # print("Index " + str(idx + 1) + " in " + str(run_epochs))
        # print("\tOutput Min:\t" + str(np.min(rnn_outputs_val)))
        # print("\tClean Min:\t" + str(np.min(clean_voice_batch[:, time_seq, :, :])))
        # print("\tOutput Max:\t" + str(np.max(rnn_outputs_val)))
        # print("\tClean Max:\t" + str(np.max(clean_voice_batch[:, time_seq, :, :])))
        # print("\tBatch Loss:\t" + str(loss_value * 32768))          # Multiplied 32768 to show the batch losses obviously.
        batchLossSum = batchLossSum + loss_value

    print("\t\tIndex " + str(idx + 1) + " Batch Loss Avg:\t" + str(batchLossSum / max_time_steps / norm_factor) + "\n")

    globalBatchLossSum = globalBatchLossSum + batchLossSum
    globalStepsSum = globalStepsSum + max_time_steps

    if (int((idx + 1) % no_of_files) == 0):
        # All batch losses sum divide global steps to get Avg
        cumulativLossAvg = globalBatchLossSum / globalStepsSum
        print("\n\t\tCumulative epochs loss Avg in latest " + str(idx + 1) + " indexes:\t" + str(cumulativLossAvg / norm_factor))
        if(cumulativLossAvg <= lastCumulativeLossAvg):
            lastCumulativeLossAvg = cumulativLossAvg            # If cumulative loss avg is smaller or equal to last avg, stay learning rate
        else:
            learning_rate = learning_rate / 5           # If cumulative loss avg is bigger than last avg, than change learning rate to 1/5
            lastCumulativeLossAvg = cumulativLossAvg
            print("\n\t\tLearning Rate changed to: " + str(learning_rate))
        globalBatchLossSum = 0          # Initialize to 0, for next indexes batch loss calculation
        globalStepsSum = 0

        os.chdir(checkpoints)
        saver.save(sess, './ssep_model_AddedLayer.ckpt', global_step=idx)
        print("\t\tSaved checkpoint\n")
        os.chdir(traindata)

os.chdir(checkpoints)
saver.save(sess, './FINAL.ckpt')
print("Saved FINAL")
sess.close()